<a href="https://colab.research.google.com/github/EmilSeyfullayev/Mercedes/blob/main/4_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!cat adc.json

{
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//0epFDzbhweLlaCgYIARAAGA4SNwF-L9IrGaIpmzTAbTapJy5ady3339W0Iuq8iFE7iUFCZZDByBhcIGt83Y7LqadpgtJx-__Xjo4",
  "revoke_uri": "https://accounts.google.com/o/oauth2/revoke",
  "scopes": [
    "openid",
    "https://www.googleapis.com/auth/userinfo.email",
    "https://www.googleapis.com/auth/cloud-platform",
    "https://www.googleapis.com/auth/appengine.admin",
    "https://www.googleapis.com/auth/compute",
    "https://www.googleapis.com/auth/accounts.reauth",
    "https://www.googleapis.com/auth/drive"
  ],
  "token_uri": "https://oauth2.googleapis.com/token",
  "type": "authorized_user"
}

In [ ]:
# shift plus enter six
!pip install -U -q PyDrive

import httplib2
import json

from google.colab import auth
from oauth2client import GOOGLE_REVOKE_URI, GOOGLE_TOKEN_URI, client
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

auth_key = {
"client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//0epFDzbhweLlaCgYIARAAGA4SNwF-L9IrGaIpmzTAbTapJy5ady3339W0Iuq8iFE7iUFCZZDByBhcIGt83Y7LqadpgtJx-__Xjo4"
}

credentials = client.OAuth2Credentials(
    access_token=None,
    client_id=auth_key['client_id'],
    client_secret=auth_key['client_secret'],
    refresh_token=auth_key['refresh_token'],
    token_expiry=None,
    token_uri=GOOGLE_TOKEN_URI,
    user_agent=None,
    revoke_uri=GOOGLE_REVOKE_URI)

credentials.refresh(httplib2.Http())
credentials.authorize(httplib2.Http())
cred = json.loads(credentials.to_json())
cred['type'] = 'authorized_user'

with open('adc.json', 'w') as outfile:
  json.dump(cred, outfile)

auth.authenticate_user()

In [ ]:
#Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())

ws = gc.open("MercedesDataBase").worksheet("sheet_1")
db_data = gd.get_as_dataframe(ws)


data = db_data

##Preprocessing
df = pd.DataFrame()

#1 #2
df['city'] = data['Şəhər']
df['car_name'] = data['Marka']

#3
model_1 = []
model_2 = []
model_3 = []

for i in data['Model']:
  model_1.append(str(i).split(" ")[0])

  try:
    model_2.append(str(i).split(" ")[1])
  except IndexError:
    model_2.append(0)

  try:
    model_3.append(str(i).split(" ")[2])
  except IndexError:
    model_3.append(0)

df['model_1'] = model_1
df['model_2'] = model_2
df['model_3'] = model_3


#4, 5, 6
df['year'] = pd.to_numeric(data['Buraxılış ili'])
df['car_type'] = data['Ban növü']
df['color'] = data['Rəng']


#7
engine_volume = []

for i in data['Mühərrik']:
  engine_volume.append(str(i).split(" L")[0])

df['engine_volume'] = pd.to_numeric(engine_volume)

#8
horse_power = []

for i in data['Mühərrikin gücü']:
  horse_power.append(str(i).split(" a.g.")[0])

df['horse_power'] = pd.to_numeric(horse_power)

#9
df['fuel_type'] = data['Yanacaq növü']

#10
went = []

for i in data['Yürüş']:
  went.append(str(i).split(" km")[0].replace(" ", ""))

df['went'] = pd.to_numeric(went)

#11 #12 #13
df['gearbox'] = data['Sürətlər qutusu']
df['transmission'] = data['Ötürücü']
df['new'] = data['Yeni']

#14 #15
currency = []
price = []

for i in data['Qiymət']:
  if str(i).split(" ")[-1] in ["AZN", "$"]:
    currency.append(str(i).split(" ")[-1])
  elif str(i).split(" ")[-1] not in ["AZN", "$"]:
    currency.append("€")

for i in data['Qiymət']:
  if str(i).split(" ")[-1] in ["AZN", "$"]:
    price.append("".join(str(i).split(" ")[0:-1]))
  elif str(i).split(" ")[-1] not in ["AZN", "$"]:
    price.append(i)

df['currency'] = currency
df['price'] = pd.to_numeric(price)

#16

date = []
time_hour = []
time_minute = []

for i in data['Tarix_Saat']:
  date.append(i.split(" ")[0])
  time_hour.append(str(i).split(" ")[1].split(":")[0])
  time_minute.append(str(i).split(" ")[1].split(":")[1])

df['date'] = pd.to_datetime(date, format="%d.%m.%Y" )
df['time_hour'] = pd.to_numeric(time_hour)
df['time_minute'] = pd.to_numeric(time_minute)

#17

values = ['products-i', 'vipped', 'featured', 'salon']

vip = []
featured = []
salon = []

for value in data['Kateqoriya']:
#########################################
  if values[1] in value:
    vip.append(1)
  elif values[1] not in value:
    vip.append(0)
########################################
for value in data['Kateqoriya']:
  if values[2] in value:
    featured.append(1)
  elif values[2] not in value:
    featured.append(0)
########################################
for value in data['Kateqoriya']:
  if values[3] in value:
    salon.append(1)
  elif values[3] not in value:
    salon.append(0)

df['vip'] = vip
df['featured'] = featured
df['salon'] = salon

#18 #19 #20 #21 #22
df['number_of-looks'] = pd.to_numeric(data['Baxışların sayı'])
df['id'] = pd.to_numeric(data['Məhsulun id-si'])
df['link'] = data['Link']
df['loan'] = data['Kredit mümkündür']
df['barter'] = data['Barter mümkündür']

#23 #24
# this one is tricky
#length of one string element (1 number is 15)
#lrngth of others (2 and more number counts) is roughly equals or less than 4 (4 numbers)

cell_phone_number = []

for i in data['Əlaqə nömrəsi']:
  if len(str(i)) > 14:
    cell_phone_number.append(i)
  else:
    cell_phone_number.append(", ".join(str(i)))

df['cell_phone_number'] = cell_phone_number
df['name_of_auto_salon'] = data['Avtosalon adı']

#25
extra_values = ['Yüngül lehimli disklər',
 'ABS', 'Lyuk', 'Yağış sensoru', 'Mərkəzi qapanma',
 'Park radarı', 'Kondisioner', 'Oturacaqların isidilməsi',
 'Dəri salon', 'Ksenon lampalar', 'Arxa görüntü kamerası',
 'Yan pərdələr', 'Oturacaqların ventilyasiyası']

alloy_wheels = [];    abs_auto_block = [];        luke = []
rain_sensors = [];    central_door_blocking = []; parking_radars = []
air_conditioner = []; seat_heating = [];          leather_interior = []
xenon_lights = [];    back_side_camera = [];      side_curtains = []
seat_ventilation = []

for i in data['Ekstra parametrlər']:
####disks
  if extra_values[0] in str(i):
    alloy_wheels.append(1)
  elif extra_values[0] not in str(i):
    alloy_wheels.append(0)

####abs
  if extra_values[1] in str(i):
    abs_auto_block.append(1)
  elif extra_values[1] not in str(i):
    abs_auto_block.append(0)


####luke
  if extra_values[2] in str(i):
    luke.append(1)
  elif extra_values[2] not in str(i):
    luke.append(0)

#rain_sensors
  if extra_values[3] in str(i):
    rain_sensors.append(1)
  elif extra_values[3] not in str(i):
    rain_sensors.append(0)

#central_door_blocking
  if extra_values[4] in str(i):
    central_door_blocking.append(1)
  elif extra_values[4] not in str(i):
    central_door_blocking.append(0)

#parking_radars
  if extra_values[5] in str(i):
    parking_radars.append(1)
  elif extra_values[5] not in str(i):
    parking_radars.append(0)

#air_conditioner
  if extra_values[6] in str(i):
    air_conditioner.append(1)
  elif extra_values[6] not in str(i):
    air_conditioner.append(0)

#seat_heating
  if extra_values[7] in str(i):
    seat_heating.append(1)
  elif extra_values[7] not in str(i):
    seat_heating.append(0)

#leather_interior
  if extra_values[8] in str(i):
    leather_interior.append(1)
  elif extra_values[8] not in str(i):
    leather_interior.append(0)

#xenon_lights
  if extra_values[9] in str(i):
    xenon_lights.append(1)
  elif extra_values[9] not in str(i):
    xenon_lights.append(0)

#back_side_camera
  if extra_values[10] in str(i):
    back_side_camera.append(1)
  elif extra_values[10] not in str(i):
    back_side_camera.append(0)

#side_curtains
  if extra_values[11] in str(i):
    side_curtains.append(1)
  elif extra_values[11] not in str(i):
    side_curtains.append(0)

#seat_ventilation
  if extra_values[12] in str(i):
    seat_ventilation.append(1)
  elif extra_values[12] not in str(i):
    seat_ventilation.append(0)


df['alloy_wheels'] = alloy_wheels;                        df['abs_auto_block'] = abs_auto_block
df['luke'] = luke;                                        df['rain_sensors'] = rain_sensors
df['central_door_blocking'] = central_door_blocking;      df['parking_radars'] = parking_radars
df['air_conditioner'] = air_conditioner;                  df['seat_heating'] = seat_heating
df['leather_interior'] = leather_interior;                df['xenon_lights'] = xenon_lights
df['back_side_camera'] = back_side_camera;                df['side_curtains'] = side_curtains
df['seat_ventilation'] = seat_ventilation

#26
df['product_text'] = data['Məhsulun mətni']

# if any(s in line for s in ('string1', 'string2', ...)): # this is to check words in text by keywords

#27
geolocation_horizontal = []
geolocation_vertical = []

for i in data['Yerləşmə məkanı']:
  try:
    geolocation_horizontal.append(float(i.split("%2C")[0]))
    geolocation_vertical.append(float(i.split("%2C")[1]))
  except AttributeError:
    geolocation_horizontal.append(i)
    geolocation_vertical.append(i)   

df['geolocation_horizontal'] = geolocation_horizontal
df['geolocation_vertical'] = geolocation_vertical

#28 #29
df['parsing_date'] = data['Parsinq olunan tarix']
df['parsing_exact_time'] = data['Parsinq olunan zaman']

price_manat_2021 = []

for i in range(len(df)):
  if df['currency'][i] == 'AZN':
    price_manat_2021.append(df['price'][i])
  elif df['currency'][i] == '$':
    price_manat_2021.append(df['price'][i]*1.7)
  elif df['currency'][i] == '€':
    price_manat_2021.append(df['price'][i]*2)
  else:
    price_manat_2021.append(0)

df['price_manat_2021'] = price_manat_2021

In [ ]:
## This cell establishes connection with sqliteDB, deletes all rows and writes new from the google sheets
import sqlite3
connection = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/GeneralParser/MercedesDataBase.db')
cursor = connection.cursor()
cursor.execute(
    '''
    DELETE FROM DataBasePrimaryTable;
    '''
)
connection.commit()
df.to_sql('DataBasePrimaryTable', connection, if_exists="replace", index=False)
data = pd.read_sql(
    'select * from DataBasePrimaryTable', 
    connection
)

In [ ]:
import sqlite3
connection = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/GeneralParser/MercedesDataBase.db')
cursor = connection.cursor()


In [ ]:
pd.read_sql_query( #same output as with read_sql (and with read_sql_table, but last is different)

    '''
    SELECT * FROM DataBasePrimaryTable WHERE car_type == 'Sedan' LIMIT 5
    ''', 

    connection
)

,index,city,car_name,model_1,model_2,model_3,year,car_type,color,engine_volume,horse_power,fuel_type,went,gearbox,transmission,new,currency,price,date,time_hour,time_minute,vip,featured,salon,number_of-looks,id,link,loan,barter,cell_phone_number,name_of_auto_salon,alloy_wheels,abs_auto_block,luke,rain_sensors,central_door_blocking,parking_radars,air_conditioner,seat_heating,leather_interior,xenon_lights,back_side_camera,side_curtains,seat_ventilation,product_text,geolocation_horizontal,geolocation_vertical,parsing_date,parsing_exact_time,price_manat_2021
0,1,Bakı,Mercedes,E,270,0,2000,Sedan,Gümüşü,2.7,170,Dizel,400000,Avtomat,Arxa,Xeyr,AZN,17500,2021-01-07 00:00:00,13,54,0,0,0,1516,4631550,https://turbo.az//autos/4631550-mercedes-e-270,0,0,"[, ]",0,1,1,1,1,1,1,1,1,1,1,1,1,1,Salam. Maşın ela veziyetdedir hecbir prablemi ...,0.0,0.0,17:56:37,2021-02-06,17500.0
1,3,Bakı,Mercedes,E,200,0,1998,Sedan,Yaş Asfalt,2.0,136,Benzin,219000,Avtomat,Arxa,Xeyr,AZN,11800,2021-01-11 00:00:00,16,11,0,0,0,442,4631571,https://turbo.az//autos/4631571-mercedes-e-200,0,0,"[, ]",0,0,1,0,0,1,0,1,0,0,0,1,0,0,No text for product,0.0,0.0,17:56:34,2021-02-06,11800.0
2,5,Bakı,Mercedes,C,230,0,1999,Sedan,Qara,2.3,193,Benzin,360000,Avtomat,Arxa,Xeyr,AZN,12500,2021-01-07 00:00:00,14,24,0,0,0,569,4631691,https://turbo.az//autos/4631691-mercedes-c-230,0,0,"[, ]",0,1,1,1,0,1,0,1,0,1,1,1,1,0,Masin ela veziyyetdedi.her seyi islek veziyyet...,0.0,0.0,17:56:32,2021-02-06,12500.0
3,6,Bakı,Mercedes,E,240,0,1998,Sedan,Gümüşü,2.4,170,Benzin,280000,Avtomat,Arxa,Xeyr,AZN,10500,2021-01-30 00:00:00,18,5,0,0,0,696,4631703,https://turbo.az//autos/4631703-mercedes-e-240,0,0,"[, ]",0,1,1,0,0,0,0,1,1,0,0,0,0,0,Masin xadavoy normal vezyetdedi\nSexsi masindi...,0.0,0.0,17:56:30,2021-02-06,10500.0
4,7,Bakı,Mercedes,E,220,0,2002,Sedan,Gümüşü,2.2,143,Dizel,368345,Avtomat,Arxa,Xeyr,AZN,17500,2021-01-07 00:00:00,14,31,0,0,0,880,4631726,https://turbo.az//autos/4631726-mercedes-e-220,0,0,"[, ]",0,1,1,0,1,1,0,1,0,1,0,1,0,0,Salam. Masin oz masinimdir oz adimadir cox yax...,0.0,0.0,17:56:29,2021-02-06,17500.0


In [ ]:
pd.read_sql_query(

    '''
    SELECT car_type, COUNT(car_type)
    FROM DataBasePrimaryTable 
    GROUP BY car_type
    ORDER BY COUNT(car_type) DESC
    ''', 

    connection
)

,car_type,COUNT(car_type)
0,Sedan,9417
1,Offroader / SUV,814
2,Universal,552
3,Van,505
4,Mikroavtobus,305
5,Yük maşını,295
6,Hetçbek / Liftbek,217
7,Minivan,142
8,Furqon,118
9,Kupe,89


In [ ]:
print("prices are divided by 1000")
pd.read_sql_query(

    '''
    SELECT 
    car_type,
    COUNT(car_type) as count_car_type,
    COUNT(car_type)/11846 as share,
    ROUND(AVG(price_manat_2021), 0)/1000 AS average_price,
    ROUND(MAX(price_manat_2021)/1000) AS max_price,
    round(MIN(price_manat_2021)/1000) AS min_price
    FROM DataBasePrimaryTable
    GROUP BY car_type
    HAVING count_car_type>100
    ORDER BY count_car_type DESC
    
    ''', 

    connection
)

prices are divided by 1000


,car_type,count_car_type,share,average_price,max_price,min_price
0,Sedan,9417,0,17.912,421.0,2.0
1,Offroader / SUV,814,0,84.748,714.0,6.0
2,Universal,552,0,13.062,201.0,4.0
3,Van,505,0,33.802,239.0,9.0
4,Mikroavtobus,305,0,22.259,131.0,5.0
5,Yük maşını,295,0,36.988,119.0,9.0
6,Hetçbek / Liftbek,217,0,12.321,48.0,4.0
7,Minivan,142,0,24.190,78.0,5.0
8,Furqon,118,0,22.804,55.0,5.0
